# Particle Filter for Diverse Driving Environments

Implementation based on Wang et al. (2020): *Online parameter estimation methods for adaptive cruise control systems*

### Setup & Initialization

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import os 

# Import scenarios
from driving_scenarios import scenario_1_non_eq, scenario_1_eq, scenario_2_data, scenario_3_data, scenario_4_data

# Import estimators - NOW INCLUDING PARTICLE FILTER
from estimators import (
    rls_filter,
    particle_filter,
    plot_parameter_convergence,
    plot_parameter_pdfs,
    plot_convergence_with_uncertainty,
    compare_rls_pf
)

### Initialize Synthetic Data

In [ ]:
np.random.seed(42) # seed for reproducibility

time = 900 # number of time steps
t_axis = np.arange(time)

dt = 0.1 # time step (10 Hz, as in the paper)

true_theta = np.array([0.08, 0.12, 1.5])

s_0 = 37.8 # initial space gap (meters)
u_0 = 33.0 # initial lead velocity (m/s)
v_0 = 32.5 # initial following velocity (m/s)

dv_max = 3.0 # maximum acceleration/deceleration (m/s^2)

print("===========================================")
print("Test parameters initialized successfully!")
print(f"Time steps: {time}, dt: {dt}")
print(f"Initial conditions: u_0 = {u_0}, v_0 = {v_0}, s_0 = {s_0}")
print(f"Theta vector initialized: α={true_theta[0]}, β={true_theta[1]}, τ={true_theta[2]}")
print(f"Total t_axis steps created: {len(t_axis)}")
print("===========================================")

## Generate Synthetic Data

In [ ]:
# Scenario 1 (non-equilibrium)
print("===========================================")
print("Scenario 1: Random Walk (Non-Equilibrium)")
print("===========================================")
u_t, v_t, s_t = scenario_1_non_eq(u_0, v_0, s_0, time, dv_max, dt, true_theta)

### Visualize the Generated Data

In [ ]:
# Plot the synthetic data
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

time_axis = np.arange(len(u_t)) * dt

# Velocity plot
axes[0].plot(time_axis, u_t, 'b-', label='Lead Vehicle (u)', linewidth=1.5)
axes[0].plot(time_axis, v_t, 'r-', label='Following Vehicle (v)', linewidth=1.5)
axes[0].set_ylabel('Velocity [m/s]')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_title('Synthetic Data - Scenario 1: Non-Equilibrium')

# Space gap plot
axes[1].plot(time_axis, s_t, 'g-', label='Space Gap (s)', linewidth=1.5)
axes[1].set_ylabel('Space Gap [m]')
axes[1].set_xlabel('Time [s]')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Particle Filter Algorithm

The particle filter estimates the augmented state $\mathbf{x}^a = [s, v, \alpha, \beta, \tau]^T$ using:

1. **State Propagation**: Propagate particles through CTH-RV dynamics
2. **Weight Update**: Update weights based on measurement likelihood
3. **Resampling**: Resample particles when effective sample size drops

### Run Particle Filter on Scenario 1 (Non-Equilibrium)

In [ ]:
# Run the particle filter
theta_history, theta_mean_history, theta_std_history, particles_snapshots = particle_filter(
    u_t, v_t, s_t, 
    dt=dt, 
    true_theta=true_theta, 
    label="Scenario 1: Non-Equilibrium",
    Np=500  # Number of particles (as in paper)
)

### Plot Parameter Convergence with Uncertainty

In [ ]:
# Plot convergence with uncertainty bands
fig = plot_convergence_with_uncertainty(
    theta_mean_history, 
    theta_std_history, 
    true_theta, 
    dt, 
    "Scenario 1: Non-Equilibrium"
)
plt.show()

### Plot Posterior PDFs at Different Time Steps

This replicates Figure 2/3 from the Wang et al. (2020) paper.

In [ ]:
# Plot posterior PDFs
fig = plot_parameter_pdfs(
    particles_snapshots, 
    true_theta, 
    dt, 
    "Scenario 1: Non-Equilibrium"
)
plt.show()

## Compare RLS vs Particle Filter

In [ ]:
# Run both methods and compare
fig, theta_rls, theta_pf, particles = compare_rls_pf(
    u_t, v_t, s_t, 
    dt=dt, 
    true_theta=true_theta, 
    label="Scenario 1: Non-Equilibrium",
    Np=500
)
plt.show()

## Scenario 1: Equilibrium Case

This demonstrates the **identifiability problem** discussed in Section III-C of the paper. At equilibrium ($u = v$), the parameters $\alpha$ and $\beta$ are not identifiable.

In [ ]:
# Generate equilibrium data
print("===========================================")
print("Scenario 1: Random Walk (Equilibrium)")
print("===========================================")
u_t_eq, v_t_eq, s_t_eq = scenario_1_eq(u_0, v_0, s_0, time, dv_max, dt, true_theta)

In [ ]:
# Compare methods on equilibrium data
fig_eq, theta_rls_eq, theta_pf_eq, particles_eq = compare_rls_pf(
    u_t_eq, v_t_eq, s_t_eq, 
    dt=dt, 
    true_theta=true_theta, 
    label="Scenario 1: Equilibrium",
    Np=500
)
plt.show()

In [ ]:
# Plot PDFs for equilibrium case - shows parameter drift
fig = plot_parameter_pdfs(
    particles_eq, 
    true_theta, 
    dt, 
    "Scenario 1: Equilibrium (Identifiability Problem)"
)
plt.show()

## Other Scenarios

### Scenario 2: Curved Road

In [ ]:
u_t_2, v_t_2, s_t_2 = scenario_2_data(u_0, v_0, s_0, time, dv_max, dt, true_theta)

fig_2, theta_rls_2, theta_pf_2, particles_2 = compare_rls_pf(
    u_t_2, v_t_2, s_t_2, 
    dt=dt, 
    true_theta=true_theta, 
    label="Scenario 2: Curved Road",
    Np=500
)
plt.show()

### Scenario 3: Suburban Environment

In [ ]:
u_t_3, v_t_3, s_t_3 = scenario_3_data(u_0, v_0, s_0, time, dv_max, dt, true_theta)

fig_3, theta_rls_3, theta_pf_3, particles_3 = compare_rls_pf(
    u_t_3, v_t_3, s_t_3, 
    dt=dt, 
    true_theta=true_theta, 
    label="Scenario 3: Suburban",
    Np=500
)
plt.show()

### Scenario 4: Aggressive Driver

In [ ]:
u_t_4, v_t_4, s_t_4 = scenario_4_data(u_0, v_0, s_0, time, dv_max, dt, true_theta)

fig_4, theta_rls_4, theta_pf_4, particles_4 = compare_rls_pf(
    u_t_4, v_t_4, s_t_4, 
    dt=dt, 
    true_theta=true_theta, 
    label="Scenario 4: Aggressive Driver",
    Np=500
)
plt.show()

## Summary Table

In [ ]:
print("\n" + "="*70)
print("SUMMARY: FINAL PARAMETER ESTIMATES")
print("="*70)
print(f"{'Scenario':<25} {'Method':<8} {'α':<10} {'β':<10} {'τ':<10}")
print("-"*70)
print(f"{'True Values':<25} {'':<8} {true_theta[0]:<10.4f} {true_theta[1]:<10.4f} {true_theta[2]:<10.4f}")
print("-"*70)

scenarios = [
    ("1: Non-Equilibrium", theta_rls, theta_pf),
    ("1: Equilibrium", theta_rls_eq, theta_pf_eq),
    ("2: Curved Road", theta_rls_2, theta_pf_2),
    ("3: Suburban", theta_rls_3, theta_pf_3),
    ("4: Aggressive", theta_rls_4, theta_pf_4),
]

for name, rls, pf in scenarios:
    print(f"{name:<25} {'RLS':<8} {rls[-1,0]:<10.4f} {rls[-1,1]:<10.4f} {rls[-1,2]:<10.4f}")
    print(f"{'':<25} {'PF':<8} {pf[-1,0]:<10.4f} {pf[-1,1]:<10.4f} {pf[-1,2]:<10.4f}")
    print("-"*70)